# 01 - Data Exploration: Italian Real Estate & Demographics

## Research Hypothesis
> "In the next 10 years, real estate in certain parts of Italy will become worthless due to demographic decline."

Italy is experiencing:
- Second highest median age in the world (after Japan)
- Fertility rate ~1.2 (well below replacement of 2.1)
- Significant rural depopulation and "borghi fantasma" (ghost towns)
- Internal migration from South to North

This notebook explores the available data to understand the relationship between demographic trends and real estate prices.

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Set display options
pd.set_option('display.max_columns', 50)
pd.set_option('display.float_format', '{:.2f}'.format)

# Paths
DATA_DIR = Path('../data')
RAW_DIR = DATA_DIR / 'raw'

print('Data directories:')
print(f'  Raw: {RAW_DIR}')
print(f'  OMI: {RAW_DIR / "omi"}')
print(f'  Geo: {RAW_DIR / "geo"}')

## 1. OMI Real Estate Data

The OMI (Osservatorio del Mercato Immobiliare) provides official real estate price quotations from Agenzia delle Entrate.

**Note:** Current dataset covers 2016-2018. For full analysis, we need to expand to 2010-2024.

In [ ]:
# Load OMI price data
omi_valori = pd.read_csv(RAW_DIR / 'omi' / 'valori.csv')

print(f'OMI Valuations: {len(omi_valori):,} rows')
print(f'Columns: {list(omi_valori.columns)}')
omi_valori.head()

In [ ]:
# Extract year and semester from filename
def parse_period(filename):
    """Extract year and semester from OMI filename."""
    parts = filename.split('_')
    period = parts[3]  # e.g., '20182' for 2018 S2
    return int(period[:4]), int(period[4])

omi_valori[['year', 'semester']] = omi_valori['file'].apply(
    lambda x: pd.Series(parse_period(x))
)

# Time period coverage
print('Time periods in dataset:')
print(omi_valori.groupby(['year', 'semester']).size().reset_index(name='count'))

In [ ]:
# Property types
print('Property types:')
print(omi_valori['Descr_Tipologia'].value_counts())

In [ ]:
# Focus on residential properties (Abitazioni civili)
residential = omi_valori[omi_valori['Descr_Tipologia'] == 'Abitazioni civili'].copy()
print(f'Residential properties: {len(residential):,} rows')

# Calculate midpoint price
residential['price_mid'] = (residential['Compr_min'] + residential['Compr_max']) / 2

# Price statistics by region
print('\nResidential price statistics by region (EUR/sqm):')
regional_prices = residential.groupby('Regione')['price_mid'].agg(['mean', 'median', 'std', 'count'])
regional_prices.sort_values('median', ascending=False)

In [ ]:
# Price distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram of prices
axes[0].hist(residential['price_mid'].dropna(), bins=50, edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Price (EUR/sqm)')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Residential Prices')
axes[0].axvline(residential['price_mid'].median(), color='red', linestyle='--', label=f'Median: {residential["price_mid"].median():.0f}')
axes[0].legend()

# Boxplot by region
top_regions = regional_prices.nlargest(10, 'count').index
residential_top = residential[residential['Regione'].isin(top_regions)]
residential_top.boxplot(column='price_mid', by='Regione', ax=axes[1], rot=45)
axes[1].set_xlabel('Region')
axes[1].set_ylabel('Price (EUR/sqm)')
axes[1].set_title('Price Distribution by Region')
plt.suptitle('')

plt.tight_layout()
plt.show()

## 2. Geographic Boundaries

Loading Italian administrative boundaries (regions, provinces, municipalities).

In [ ]:
# Load geographic data
regions = gpd.read_file(RAW_DIR / 'geo' / 'regioni.geojson')
provinces = gpd.read_file(RAW_DIR / 'geo' / 'province.geojson')
comuni = gpd.read_file(RAW_DIR / 'geo' / 'comuni.geojson')

print(f'Regions: {len(regions)}')
print(f'Provinces: {len(provinces)}')
print(f'Municipalities (comuni): {len(comuni)}')

print('\nRegion columns:', list(regions.columns))
print('\nMunicipality columns:', list(comuni.columns))

In [ ]:
# Map of Italy with regions
fig, ax = plt.subplots(1, 1, figsize=(10, 12))
regions.plot(ax=ax, edgecolor='black', linewidth=0.5, cmap='Set3')
ax.set_title('Italian Regions')
ax.axis('off')
plt.show()

In [ ]:
# Create a price map by aggregating to municipality level
# First, get average price per municipality
avg_prices = residential.groupby('Comune_ISTAT')['price_mid'].mean().reset_index()
avg_prices.columns = ['comune_istat', 'avg_price']

print(f'Municipalities with price data: {len(avg_prices)}')
avg_prices.head()

## 3. Regional Price Analysis

Examining price patterns across Italy's geographic regions.

In [ ]:
# Aggregate prices to region level for mapping
region_prices = residential.groupby('Regione').agg({
    'price_mid': ['mean', 'median', 'count'],
    'Comune_ISTAT': 'nunique'
}).round(0)
region_prices.columns = ['mean_price', 'median_price', 'num_records', 'num_comuni']
region_prices = region_prices.reset_index()

print('Price statistics by region:')
region_prices.sort_values('median_price', ascending=False)

In [ ]:
# Price correlation with geographic position (North-South)
# Join prices with geographic centroids
regions_with_prices = regions.copy()
regions_with_prices['centroid_lat'] = regions_with_prices.geometry.centroid.y

# Map region names (need to standardize)
name_mapping = {
    'LOMBARDIA': 'Lombardia',
    'PIEMONTE': 'Piemonte',
    'VENETO': 'Veneto',
    'EMILIA-ROMAGNA': 'Emilia-Romagna',
    'TOSCANA': 'Toscana',
    'LAZIO': 'Lazio',
    'CAMPANIA': 'Campania',
    'SICILIA': 'Sicilia',
    'PUGLIA': 'Puglia',
    'CALABRIA': 'Calabria',
}

print('Region names in OMI data:', residential['Regione'].unique()[:10])
print('Region names in GeoJSON:', regions['reg_name'].unique()[:10] if 'reg_name' in regions.columns else regions.columns)

## 4. Key Findings & Next Steps

### Findings from Initial Exploration

1. **Data Coverage**: Current OMI dataset covers 2016-2018 (limited for trend analysis)
2. **Price Variation**: Significant price variation across regions (North typically higher)
3. **Municipality Coverage**: ~8,000 municipalities in geographic data

### Data Gaps

- **Missing**: ISTAT demographic data (population, age distribution, migration)
- **Limited**: OMI data only covers 2016-2018 (need 2010-2024 for trend analysis)

### Next Steps

1. **Download ISTAT demographic data** from demo.istat.it
2. **Expand OMI data** to cover more years
3. **Merge datasets** on ISTAT municipality codes
4. **Analyze demographic trends** vs price changes

In [ ]:
# Summary statistics
print('='*60)
print('DATA SUMMARY')
print('='*60)
print(f'OMI Records: {len(omi_valori):,}')
print(f'Residential Records: {len(residential):,}')
print(f'Time Period: 2016-2018 (6 semesters)')
print(f'Regions: {residential["Regione"].nunique()}')
print(f'Provinces: {residential["Prov"].nunique()}')
print(f'Municipalities: {residential["Comune_ISTAT"].nunique()}')
print(f'\nMedian Residential Price: {residential["price_mid"].median():,.0f} EUR/sqm')
print(f'Mean Residential Price: {residential["price_mid"].mean():,.0f} EUR/sqm')
print('='*60)